# VAR Forecasting for Multiple Datasets (with Extended Metrics)
Includes MSE (Test Loss), MAE, Best Validation Loss, and Best Test Loss.


In [1]:

import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import zscore


def run_var_model_extended(df, forecast_len, maxlags=15):
    df = df.dropna().select_dtypes(include=[np.number])
    df = df.apply(zscore)  # Standardize features
    if len(df) <= forecast_len + maxlags:
        raise ValueError("Not enough data for VAR with given forecast_len.")
    model = VAR(df)
    results = model.fit(maxlags=maxlags, ic='aic')
    lag_order = results.k_ar
    input_data = df.values[-lag_order:]
    forecast = results.forecast(input_data, steps=forecast_len)

    val_data = df.values[-forecast_len*2:-forecast_len]
    pred_val = results.forecast(df.values[-forecast_len*2 - lag_order:-forecast_len].reshape(-1, df.shape[1]), steps=forecast_len)
    best_vali_loss = mean_squared_error(val_data, pred_val)

    true_vals = df.values[-forecast_len:]
    best_test_loss = mean_squared_error(true_vals, forecast)
    mae = mean_absolute_error(true_vals, forecast)

    return forecast, best_test_loss, mae, best_vali_loss